In [1]:
import torch
import os
import numpy as np
import random
from tqdm import tqdm
from sys import stdout

In [2]:
MAIN_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(MAIN_DIR, "Solid_droplet", "Data")

In [3]:
from DataLoader import *

In [4]:
# Load Data
data = DataLoader(DATA_DIR, train_frac=0.6, val_frac=0.2, test_frac=0.2, use_seed=True, seed_val=42)

# Load Data from DataLoader
train_image = data.train
val_image = data.val
test_image = data.test
train_labels = data.train_label
val_labels = data.val_label
test_labels = data.test_label

d:\University\Master\MY1\Q4\4AI000\4AI000\Model\DataLoader.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  train_images = tensor(self._get_images(train_split, DATA_DIR))


In [5]:
class Model(torch.nn.Module):
    def __init__(self, device, dtype) -> None:
        """Initialisation of the Model class"""
        super().__init__()
        self.cnn1 = torch.nn.Conv2d(in_channels = 1, out_channels=32, kernel_size=3, padding=2, device=device, dtype=dtype)
        self.cnn2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2, device=device, dtype=dtype)
        self.max1 = torch.nn.MaxPool2d(kernel_size=(2,2))
        self.cnn3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, padding=2, device=device, dtype=dtype)
        self.cnn4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=2, device=device, dtype=dtype)
        self.max2 = torch.nn.MaxPool2d(kernel_size=(2,2))
        self.flat = torch.nn.Flatten()
        self.lay1 = torch.nn.Linear(3136, 1024, device=device, dtype=dtype).double()
        self.relu = torch.nn.ReLU()
        self.lay2 = torch.nn.Linear(1024, 3, device=device, dtype=dtype).double()
    
    def forward(self, x):
        """Forward pass of the model"""
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.max1(x)
        x = self.cnn3(x)
        x = self.cnn4(x)
        x = self.max2(x)
        x = self.flat(x)
        x = self.lay1(x)
        x = self.relu(x)
        y = self.lay2(x)
        return y[:,0]


In [6]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# Get the model
model = Model(device=device, dtype=dtype)

# Set the optimizer
optimizer = torch.optim.Adam(model.parameters())
loss = torch.nn.MSELoss()

input = train_image
target = train_labels
output = loss(input, target)

# Set up data
epochs = 5
losses = []

for i in range(epochs):
    batch_loss = []
    for j, data in enumerate(dl):
        optimizer.zero_grad()
        ypred = model(data)
        
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
    if i>10: 
        optimizer.lr = 0.0005
    losses .append(sum(batch_loss) / len(batch_loss))
    print('Epoch {}:\tloss {:.4f}'.format(i, losses [-1]))

c:\Users\20192707\Anaconda3\envs\5aua0\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([27, 3])) that is different to the input size (torch.Size([27, 320, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (256) must match the size of tensor b (3) at non-singleton dimension 2

In [ ]:
def train_model (Model, train_loader:list, val_loader:list, epochs:int=50_000, learning_rate:float=0.001):
    """Training function"""
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    
    model = Model()

    #train_losses = []
    #val_losses = []
    
    # Loss and optimizer
    #optimizer = torch.optim.Adam(model.parameters()) 
    #Xtrain, Xval, Ytrain, Yval = [torch.as_tensor(x) for x in [Xtrain, Xval, Ytrain, Yval]] 
    #for epoch in range(epochs): #a)
    #    Loss = torch.mean((model(Xtrain)-Ytrain)**2) 
    #    optimizer.zero_grad() 
    #    Loss.backward() 
    #    optimizer.step() 
    #    if epoch%1000==0: 
    #        print(epoch,Loss.item())
    #        train_losses.append(Loss.item())
    #        val_losses.append(torch.mean((model(Xval)-Yval)**2).item())


In [ ]:
train_image[0].size()

In [ ]:
dl = torch.utils.data.DataLoader(train_image, batch_size=1, shuffle=False)


In [ ]:
torch.utils.data.DataLoader(